#LAB 06 Programação Paralela OpenACC


In [ ]:
##Jonatas de Brito Silva - 10403674
##Felipe Gyotoku Koike - 10409640
##Bruno Viana Tripoli Barbosa - 10409547

## Baixando e instalando pacotes HPC SDK da NVidia

In [ ]:
!wget https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9_20.9_amd64.deb

--2024-11-16 13:55:28--  https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9_20.9_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2117119120 (2.0G) [application/x-deb]
Saving to: ‘nvhpc-20-9_20.9_amd64.deb’

nvhpc-20-9_20.9_amd 100%[===================>]   1.97G   104MB/s    in 20s     

2024-11-16 13:55:48 (102 MB/s) - ‘nvhpc-20-9_20.9_amd64.deb’ saved [2117119120/2117119120]



In [ ]:
!wget https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-2020_20.9_amd64.deb

--2024-11-16 13:55:58--  https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-2020_20.9_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272 (1.2K) [application/x-deb]
Saving to: ‘nvhpc-2020_20.9_amd64.deb’

nvhpc-2020_20.9_amd 100%[===================>]   1.24K  --.-KB/s    in 0s      

2024-11-16 13:55:58 (62.9 MB/s) - ‘nvhpc-2020_20.9_amd64.deb’ saved [1272/1272]



In [ ]:
!wget https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9-cuda-multi_20.9_amd64.deb

--2024-11-16 13:56:01--  https://developer.download.nvidia.com/hpc-sdk/20.9/nvhpc-20-9-cuda-multi_20.9_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1558347920 (1.5G) [application/x-deb]
Saving to: ‘nvhpc-20-9-cuda-multi_20.9_amd64.deb’

nvhpc-20-9-cuda-mul 100%[===================>]   1.45G   101MB/s    in 15s     

2024-11-16 13:56:17 (96.2 MB/s) - ‘nvhpc-20-9-cuda-multi_20.9_amd64.deb’ saved [1558347920/1558347920]



In [ ]:
!sudo apt-get install ./nvhpc-20-9_20.9_amd64.deb ./nvhpc-2020_20.9_amd64.deb ./nvhpc-20-9-cuda-multi_20.9_amd64.deb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvhpc-20-9' instead of './nvhpc-20-9_20.9_amd64.deb'
Note, selecting 'nvhpc-2020' instead of './nvhpc-2020_20.9_amd64.deb'
Note, selecting 'nvhpc-20-9-cuda-multi' instead of './nvhpc-20-9-cuda-multi_20.9_amd64.deb'
nvhpc-20-9 is already the newest version (20.9).
nvhpc-20-9-cuda-multi is already the newest version (20.9).
nvhpc-2020 is already the newest version (20.9).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
2 not fully installed or removed.
After this operation, 0 B of additional disk space will be used.
Setting up nvidia-dkms-525 (525.147.05-0ubuntu1) ...

A modprobe blacklist file has been created at /etc/modprobe.d to prevent Nouveau
from loading. This can be reverted by deleting the following file:
/etc/modprobe.d/nvidia-graphics-drivers.conf

A new initrd image has also been created. To revert, please regenerate your
initrd by running the foll

In [ ]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/20.9/compilers/bin/nvc --version


nvc 20.9-0 LLVM 64-bit target on x86-64 Linux -tp haswell 
NVIDIA Compilers and Tools
Copyright (c) 2020, NVIDIA CORPORATION.  All rights reserved.


In [ ]:
!nvidia-smi

Sat Nov 16 14:38:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

##Exercício 01: Regra do Trapézio usando OpenACC

In [5]:
%%writefile integralTrapezio.c
#include <stdio.h>
#include <math.h>

double f(double x) {
    return x * x;
}

double trapezoidal_rule_openacc(double a, double b, int n) {
    double h = (b - a) / n;
    double integral = (f(a) + f(b)) / 2.0;
    double sum = 0.0;

    #pragma acc parallel loop reduction(+:sum)
    for (int i = 1; i < n; i++) {
        double x_i = a + i * h;
        sum += f(x_i);
    }

    integral += sum;
    integral *= h;
    return integral;
}

int main() {
    double a = 0.0, b = 1.0;
    int n = 100000;
    printf("Integral: %f\n", trapezoidal_rule_openacc(a, b, n));
    return 0;
}


Writing integralTrapezio.c


In [8]:
!gcc -fopenacc integralTrapezio.c -o integralTrapezio
!./integralTrapezio

Integral: 0.333333


##Exercício 02: Nova Multiplicação de Matrizes OpenAcc

In [9]:
%%writefile mult_openAcc.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define MAX_VAL 255

// Função para alocar uma matriz dinamicamente
float **allocate_matrix(int rows, int cols) {
    float **matrix = (float **)malloc(rows * sizeof(float *));
    for (int i = 0; i < rows; i++) {
        matrix[i] = (float *)malloc(cols * sizeof(float));
    }
    return matrix;
}

// Função para liberar uma matriz
void free_matrix(float **matrix, int rows) {
    for (int i = 0; i < rows; i++) {
        free(matrix[i]);
    }
    free(matrix);
}

// Função para inicializar uma matriz com valores aleatórios
void initialize_matrix(float **matrix, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            matrix[i][j] = rand() % (MAX_VAL + 1);
        }
    }
}

// Função para aplicar a convolução
void convolve(float **input, float **output, float **kernel, int rows, int cols, int kernel_size) {
    int offset = kernel_size / 2;

    // Iterar pelos elementos da matriz de saída
    for (int i = offset; i < rows - offset; i++) {
        for (int j = offset; j < cols - offset; j++) {
            float sum = 0.0;

            // Aplicar o kernel
            for (int ki = 0; ki < kernel_size; ki++) {
                for (int kj = 0; kj < kernel_size; kj++) {
                    int x = i + ki - offset;
                    int y = j + kj - offset;
                    sum += input[x][y] * kernel[ki][kj];
                }
            }

            output[i - offset][j - offset] = sum;
        }
    }
}

// Função para criar um kernel
void initialize_kernel(float **kernel, int size) {
    for (int i = 0; i < size; i++) {
        for (int j = 0; j < size; j++) {
            kernel[i][j] = 1.0 / (size * size); // Média
        }
    }
}

int main() {
    srand(time(NULL));

    // Tamanhos das matrizes
    int sizes[] = {1048, 4096};
    int kernel_sizes[] = {5, 11, 33};

    for (int s = 0; s < 2; s++) { // Para cada tamanho de matriz
        int rows = sizes[s], cols = sizes[s];

        // Alocar e inicializar matriz original
        float **input = allocate_matrix(rows, cols);
        initialize_matrix(input, rows, cols);

        for (int k = 0; k < 3; k++) { // Para cada tamanho de kernel
            int kernel_size = kernel_sizes[k];
            int output_rows = rows - kernel_size + 1;
            int output_cols = cols - kernel_size + 1;

            // Alocar matriz de saída e kernel
            float **output = allocate_matrix(output_rows, output_cols);
            float **kernel = allocate_matrix(kernel_size, kernel_size);

            initialize_kernel(kernel, kernel_size);

            // Medir o tempo da convolução
            clock_t start = clock();
            convolve(input, output, kernel, rows, cols, kernel_size);
            clock_t end = clock();

            printf("Convolução: Matriz %dx%d com Kernel %dx%d -> Tempo: %.2f segundos\n",
                   rows, cols, kernel_size, kernel_size, (double)(end - start) / CLOCKS_PER_SEC);

            // Liberar memória da matriz de saída e kernel
            free_matrix(output, output_rows);
            free_matrix(kernel, kernel_size);
        }

        // Liberar memória da matriz de entrada
        free_matrix(input, rows);
    }

    return 0;
}


Writing mult_openAcc.c


In [10]:
!gcc -fopenacc mult_openAcc.c -o mult_openAcc
!./mult_openAcc

Convolução: Matriz 1048x1048 com Kernel 5x5 -> Tempo: 0.14 segundos
Convolução: Matriz 1048x1048 com Kernel 11x11 -> Tempo: 0.64 segundos
Convolução: Matriz 1048x1048 com Kernel 33x33 -> Tempo: 5.55 segundos
Convolução: Matriz 4096x4096 com Kernel 5x5 -> Tempo: 2.97 segundos
Convolução: Matriz 4096x4096 com Kernel 11x11 -> Tempo: 10.10 segundos
Convolução: Matriz 4096x4096 com Kernel 33x33 -> Tempo: 97.39 segundos
